In [ ]:
pip install pandas pyarrow fastparquet openpyxl


   ---------------------------------------- 0.0/673.3 kB ? eta -:--:--
   --------------- ------------------------ 262.1/673.3 kB ? eta -:--:--
   ---------------------------------------- 673.3/673.3 kB 6.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 17.5 MB/s eta 0:00:00

   ---------------------------------------- 2/2 [fastparquet]

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Step 0 - Imports and paths
import pandas as pd
import numpy as np
import re

SRC = "Main.csv"                # input
DST = "Main_clean.csv"          # cleaned output
DICT = "Main_data_dictionary.csv"


In [8]:
# Step 1 - Load everything as string to avoid mixed-type surprises
na_tokens = ["", "NA", "NaN", "null", "None", "N/A"]
df_raw = pd.read_csv(
    SRC,
    dtype=str,
    keep_default_na=True,
    na_values=na_tokens
)

# Make a working copy - do not drop rows
df = df_raw.copy()
print("Rows x Cols:", df.shape)


Rows x Cols: (187260, 30)


In [9]:
# Step 2 - Whitespace normalization on all cells, preserve content
def normalize_str_col(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    # keep blank as <NA> if truly empty after trim
    s = s.replace({"": pd.NA})
    return s

for c in df.columns:
    df[c] = normalize_str_col(df[c])


In [10]:
# Step 3 - Clean column names for Power BI
# - remove leading "Graduate_School_Dashboard_Dataset_"
# - replace '_' with ' '
# - title case
# - ensure uniqueness

def clean_colname(name: str) -> str:
    name = re.sub(r"^Graduate_School_Dashboard_Dataset_", "", name)
    name = name.replace("_", " ")
    name = name.strip()
    name = re.sub(r"\s+", " ", name)
    return name.title()

new_names = [clean_colname(c) for c in df.columns]

# ensure uniqueness
seen = {}
final_names = []
for n in new_names:
    if n not in seen:
        seen[n] = 1
        final_names.append(n)
    else:
        seen[n] += 1
        final_names.append(f"{n} {seen[n]}")

rename_map = dict(zip(df.columns, final_names))
df.rename(columns=rename_map, inplace=True)

print("Cleaned column names:")
print(list(df.columns))


Cleaned column names:
['Event', 'Term Code', 'Calendar Year', 'Academic Year', 'Fiscal Year', 'Degree Type', 'Degree Level Code', 'Degree Level', 'Degree', 'Program Code', 'Program', 'Major Code', 'Major', 'College Code', 'College', 'Department Code', 'Department', 'Campus Code', 'Campus', 'Gender Code', 'Gender', 'Race And Ethnicity Code', 'Race And Ethnicity', 'Citizenship Status Code', 'Citizensihp Status', 'Residency Code', 'Residency', 'Time Status Code', 'Time Status', 'Total']


In [11]:
# Step 4 - Protect identifiers and format Term Code
# Treat columns containing these tokens as identifiers to keep as text
ID_TOKENS = ("code", "id", "uid", "guid")

# Identify Term Code columns after renaming (e.g., "Term Code")
term_code_cols = [c for c in df.columns if re.search(r"\bterm\s*code\b", c, flags=re.I)]

# Left-pad Term Code to 6 digits, but keep as text
for c in term_code_cols:
    df[c] = df[c].astype("string")
    df[c] = df[c].where(df[c].isna(), df[c].str.replace(r"\D", "", regex=True))  # strip non-digits only if present
    df[c] = df[c].where(df[c].isna(), df[c].str.zfill(6))


In [12]:
# Step 5 - Infer numeric-like columns safely while preserving ID/code fields
def is_identifier(colname: str) -> bool:
    return any(tok in colname.lower() for tok in ID_TOKENS)

NUMERIC_THRESHOLD = 0.80  # at least 80 percent of non-null values must be numeric to coerce

numeric_like = []
for c in df.columns:
    if is_identifier(c):
        continue
    s = df[c]
    if s.isna().all():
        continue
    # Attempt numeric coercion
    coerced = pd.to_numeric(s, errors="coerce")
    non_null = s.notna().sum()
    numeric_ratio = coerced.notna().sum() / max(1, non_null)
    if numeric_ratio >= NUMERIC_THRESHOLD:
        numeric_like.append(c)

print("Numeric-like columns inferred:", numeric_like[:10], "..." if len(numeric_like) > 10 else "")



Numeric-like columns inferred: ['Calendar Year', 'Academic Year', 'Fiscal Year', 'Total'] 


In [13]:
# Step 6 - Cast numeric-like columns and fill missing numerics with 0
for c in numeric_like:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Explicit handling for Year columns - keep numeric
year_cols = [c for c in df.columns if re.search(r"\byear\b", c, flags=re.I)]
for c in year_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Fill numeric NaN with 0 and downcast to Int64 if values are whole numbers
for c in numeric_like + year_cols:
    if c in df.columns:
        ser = df[c]
        ser = ser.fillna(0)
        # Downcast to Int64 if all values are whole numbers
        as_int = pd.Series(np.floor(ser) == ser)
        if bool(as_int.min()):  # all True
            df[c] = ser.astype("Int64")
        else:
            df[c] = ser.astype("Float64")

# For all remaining non-numeric columns, replace missing with empty string
for c in df.columns:
    if c not in numeric_like and c not in year_cols:
        df[c] = df[c].astype("string").fillna("")


In [14]:
# Step 7 - Assertions to guarantee row preservation and no NaN numerics
assert df.shape[0] == df_raw.shape[0], "Row count changed - not allowed"
nan_numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c]) and df[c].isna().any()]
assert len(nan_numeric_cols) == 0, f"Numeric cols with NaN remain: {nan_numeric_cols}"

print("Rows preserved and numeric NaNs filled with 0.")


Rows preserved and numeric NaNs filled with 0.


In [15]:
# Step 8 - Optional data dictionary for documentation
data_dict = pd.DataFrame({
    "Column": df.columns,
    "Dtype": [str(df[c].dtype) for c in df.columns],
    "NonNull": [int(df[c].notna().sum()) for c in df.columns],
    "Null": [int(df[c].isna().sum()) for c in df.columns],
    "Distinct": [int(df[c].nunique(dropna=True)) for c in df.columns]
})
data_dict.to_csv(DICT, index=False, encoding="utf-8-sig")
print("Wrote data dictionary:", DICT)


Wrote data dictionary: Main_data_dictionary.csv


In [16]:
# Step 9 - Export cleaned dataset for Power BI
# UTF-8 with BOM avoids odd character issues in some Windows tools
df.to_csv(DST, index=False, encoding="utf-8-sig")
print("Wrote cleaned file:", DST, "| shape:", df.shape)


Wrote cleaned file: Main_clean.csv | shape: (187260, 30)
